In [ ]:
import cv2
import networkx as nx
import numpy as np
import random
import matplotlib.pyplot as plt

class Node:
    def _init_(self):
        self.infected = False

class Graph:
    def _init_(self, node_num):
        self.nodes = [Node() for _ in range(node_num)]
        self.edges = [[0 for _ in range(node_num)] for _ in range(node_num)]

def read_graph_from_file(file_name):
    with open(file_name, 'r') as file:
        lines = file.readlines()

    node_num = int(lines[0].strip())
    graph = Graph(node_num)

    for i in range(1, len(lines)):
        values = list(map(int, lines[i].strip().split()))
        for j in range(len(values)):
            graph.edges[i - 1][j] = values[j]

    return graph

def create_adjacency_matrix_from_image(image_path):
    try:
        image = cv2.imread(image_path, 0)
        if image is None:
            print(f"Error: Unable to load image at {image_path}")
            return None, None

        cv2.imshow('Image', image)
        cv2.waitKey(0)
        _, thresholded = cv2.threshold(image, 200, 255, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        G = nx.Graph()

        for contour in contours:
            for i in range(len(contour)):
                node1 = tuple(contour[i][0])
                node2 = tuple(contour[(i + 1) % len(contour)][0])
                G.add_edge(node1, node2)

        adjacency_matrix = nx.adjacency_matrix(G).todense()
        return G, adjacency_matrix
    except Exception as e:
        print(f"Error: {e}")
        return None, None

def independent_cascade(graph, seed_nodes, infection_prob, node_num):
    new_infect = [0] * node_num

    for node in seed_nodes:
        graph.nodes[node].infected = True

    while True:
        any_new_infections = False

        for i in range(node_num):
            if graph.nodes[i].infected:
                for j in range(node_num):
                    if not graph.nodes[j].infected and graph.edges[i][j]:
                        random_value = random.random()
                        if random_value < infection_prob:
                            new_infect[j] = 1
                            any_new_infections = True

        if not any_new_infections:
            break

        for i in range(node_num):
            if new_infect[i]:
                graph.nodes[i].infected = True
                new_infect[i] = 0

def visualize_graph(graph, data_loss_nodes):
    G = nx.Graph(graph.edges)
    pos = nx.spring_layout(G)

    nx.draw(G, pos, with_labels=True, node_color='skyblue', node_size=1000, font_size=10, font_color='black')
    nx.draw_networkx_nodes(G, pos, nodelist=data_loss_nodes, node_color='red', node_size=1000)

    plt.title("Graph Visualization with Data Loss Nodes")
    plt.show()

def main():
    file_or_image = input("Enter 'file' to read graph from a file or 'image' to read graph from an image: ")

    if file_or_image.lower() == 'file':
        file_name = input("Enter the name of the file containing the graph: ")
        graph = read_graph_from_file(file_name)
        if not graph:
            print("Error: Unable to create the graph. Please check the input file.")
            return
        
    elif file_or_image.lower() == 'image':
        image_path = input("Enter the path to the image containing the graph: ")
        graph, adjacency_matrix = create_adjacency_matrix_from_image(image_path)
        if graph is None or adjacency_matrix is None:
            print("Error: Unable to create the graph from the image.")
            return
        # Create a Graph object based on the adjacency matrix (if necessary)
        # graph = create_graph_from_adjacency_matrix(adjacency_matrix)
    else:
        print("Invalid choice. Please enter 'file' or 'image'.")
        return

    node_num = len(graph.nodes)
    seed_nodes = []  # Add code to input seed nodes if needed
    infection_prob = float(input("Enter the infection probability (0 to 1):"))
    if not 0 <= infection_prob <= 1:
        print("Invalid infection probability. Please enter a valid probability between 0 and 1.")
        return
    
    # Run independent cascade simulation
    independent_cascade(graph, seed_nodes, infection_prob, node_num)

    # Calculate data loss percentage and get data loss nodes
    total_nodes = node_num
    infected_nodes = sum(node.infected for node in graph.nodes)
    data_loss_nodes = [i for i, node in enumerate(graph.nodes) if not node.infected]
    data_loss_percentage = ((total_nodes - infected_nodes) / total_nodes) * 100.0

    print(f"Total Nodes: {total_nodes}")
    print(f"Infected Nodes: {infected_nodes}")
    print(f"Data Loss Percentage: {data_loss_percentage:.2f}%")
    print("Edges:")
    for i in range(node_num):
        for j in range(node_num):
            if graph.edges[i][j]:
                print(f"Edge: {i} -> {j}")


    # Visualize the graph with data loss nodes
    visualize_graph(graph, data_loss_nodes)

if _name_ == "_main_":
    main()

In [ ]:
!pip install opencv-python


In [ ]:
import cv2
import networkx as nx
import numpy as np
import random
import matplotlib.pyplot as plt

class Node:
    def _init_(self):
        self.infected = False

class Graph:
    def _init_(self, node_num):
        self.nodes = [Node() for _ in range(node_num)]
        self.edges = [[0 for _ in range(node_num)] for _ in range(node_num)]

def read_graph_from_file(file_name):
    with open(file_name, 'r') as file:
        lines = file.readlines()

    node_num = int(lines[0].strip())
    graph = Graph(node_num)

    for i in range(1, len(lines)):
        values = list(map(int, lines[i].strip().split()))
        for j in range(len(values)):
            graph.edges[i - 1][j] = values[j]

    return graph

def create_adjacency_matrix_from_image(image_path):
    try:
        image = cv2.imread(image_path, 0)
        if image is None:
            print(f"Error: Unable to load image at {image_path}")
            return None, None

        cv2.imshow('Image', image)
        cv2.waitKey(0)
        _, thresholded = cv2.threshold(image, 200, 255, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        G = nx.Graph()

        for contour in contours:
            for i in range(len(contour)):
                node1 = tuple(contour[i][0])
                node2 = tuple(contour[(i + 1) % len(contour)][0])
                G.add_edge(node1, node2)

        adjacency_matrix = nx.adjacency_matrix(G).todense()
        return G, adjacency_matrix
    except Exception as e:
        print(f"Error: {e}")
        return None, None

def independent_cascade(graph, seed_nodes, infection_prob, node_num):
    new_infect = [0] * node_num

    for node in seed_nodes:
        graph.nodes[node].infected = True

    while True:
        any_new_infections = False

        for i in range(node_num):
            if graph.nodes[i].infected:
                for j in range(node_num):
                    if not graph.nodes[j].infected and graph.edges[i][j]:
                        random_value = random.random()
                        if random_value < infection_prob:
                            new_infect[j] = 1
                            any_new_infections = True

        if not any_new_infections:
            break

        for i in range(node_num):
            if new_infect[i]:
                graph.nodes[i].infected = True
                new_infect[i] = 0

def visualize_graph(graph, data_loss_nodes):
    G = nx.Graph(graph.edges)
    pos = nx.spring_layout(G)

    nx.draw(G, pos, with_labels=True, node_color='skyblue', node_size=1000, font_size=10, font_color='black')
    nx.draw_networkx_nodes(G, pos, nodelist=data_loss_nodes, node_color='red', node_size=1000)

    plt.title("Graph Visualization with Data Loss Nodes")
    plt.show()

def main():
    file_or_image = input("Enter 'file' to read graph from a file or 'image' to read graph from an image: ")

    if file_or_image.lower() == 'file':
        file_name = input("Enter the name of the file containing the graph: ")
        graph = read_graph_from_file(file_name)
        if not graph:
            print("Error: Unable to create the graph. Please check the input file.")
            return
        
    elif file_or_image.lower() == 'image':
        image_path = input("Enter the path to the image containing the graph: ")
        graph, adjacency_matrix = create_adjacency_matrix_from_image(image_path)
        if graph is None or adjacency_matrix is None:
            print("Error: Unable to create the graph from the image.")
            return
        # Create a Graph object based on the adjacency matrix (if necessary)
        # graph = create_graph_from_adjacency_matrix(adjacency_matrix)
    else:
        print("Invalid choice. Please enter 'file' or 'image'.")
        return

    node_num = len(graph.nodes)
    seed_nodes = []  # Add code to input seed nodes if needed
    infection_prob = float(input("Enter the infection probability (0 to 1):"))
    if not 0 <= infection_prob <= 1:
        print("Invalid infection probability. Please enter a valid probability between 0 and 1.")
        return
    
    # Run independent cascade simulation
    independent_cascade(graph, seed_nodes, infection_prob, node_num)

    # Calculate data loss percentage and get data loss nodes
    total_nodes = node_num
    infected_nodes = sum(node.infected for node in graph.nodes)
    data_loss_nodes = [i for i, node in enumerate(graph.nodes) if not node.infected]
    data_loss_percentage = ((total_nodes - infected_nodes) / total_nodes) * 100.0

    print(f"Total Nodes: {total_nodes}")
    print(f"Infected Nodes: {infected_nodes}")
    print(f"Data Loss Percentage: {data_loss_percentage:.2f}%")
    print("Edges:")
    for i in range(node_num):
        for j in range(node_num):
            if graph.edges[i][j]:
                print(f"Edge: {i} -> {j}")


    # Visualize the graph with data loss nodes
    visualize_graph(graph, data_loss_nodes)

if __name__ == "_main_":
    main()

In [ ]:
import cv2
import networkx as nx
import numpy as np
import random
import matplotlib.pyplot as plt

class Node:
    def _init_(self):
        self.infected = False

class Graph:
    def _init_(self, node_num):
        self.nodes = [Node() for _ in range(node_num)]
        self.edges = [[0 for _ in range(node_num)] for _ in range(node_num)]

def read_graph_from_file(file_name):
    with open(file_name, 'r') as file:
        lines = file.readlines()

    node_num = int(lines[0].strip())
    graph = Graph(node_num)

    for i in range(1, len(lines)):
        values = list(map(int, lines[i].strip().split()))
        for j in range(len(values)):
            graph.edges[i - 1][j] = values[j]

    return graph

def create_adjacency_matrix_from_image(image_path):
    try:
        print("Attempting to load image...")
        image = cv2.imread(image_path, 0)
        if image is None:
            print(f"Error: Unable to load image at {image_path}")
            return None, None

        cv2.imshow('Image', image)
        cv2.waitKey(0)
        _, thresholded = cv2.threshold(image, 200, 255, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        G = nx.Graph()

        for contour in contours:
            for i in range(len(contour)):
                node1 = tuple(contour[i][0])
                node2 = tuple(contour[(i + 1) % len(contour)][0])
                G.add_edge(node1, node2)

        adjacency_matrix = nx.adjacency_matrix(G).todense()
        return G, adjacency_matrix
    except Exception as e:
        print(f"Error: {e}")
        return None, None

def independent_cascade(graph, seed_nodes, infection_prob, node_num):
    new_infect = [0] * node_num

    for node in seed_nodes:
        graph.nodes[node].infected = True

    while True:
        any_new_infections = False

        for i in range(node_num):
            if graph.nodes[i].infected:
                for j in range(node_num):
                    if not graph.nodes[j].infected and graph.edges[i][j]:
                        random_value = random.random()
                        if random_value < infection_prob:
                            new_infect[j] = 1
                            any_new_infections = True

        if not any_new_infections:
            break

        for i in range(node_num):
            if new_infect[i]:
                graph.nodes[i].infected = True
                new_infect[i] = 0

def visualize_graph(graph, data_loss_nodes):
    G = nx.Graph(graph.edges)
    pos = nx.spring_layout(G)

    nx.draw(G, pos, with_labels=True, node_color='skyblue', node_size=1000, font_size=10, font_color='black')
    nx.draw_networkx_nodes(G, pos, nodelist=data_loss_nodes, node_color='red', node_size=1000)

    plt.title("Graph Visualization with Data Loss Nodes")
    plt.show()

def main():
    print("Starting main function...")
    file_or_image = input("Enter 'file' to read graph from a file or 'image' to read graph from an image: ")

    if file_or_image.lower() == 'file':
        file_name = input("Enter the name of the file containing the graph: ")
        graph = read_graph_from_file(file_name)
        if not graph:
            print("Error: Unable to create the graph. Please check the input file.")
            return
        
    elif file_or_image.lower() == 'image':
        image_path = input("Enter the path to the image containing the graph: ")
        graph, adjacency_matrix = create_adjacency_matrix_from_image(image_path)
        if graph is None or adjacency_matrix is None:
            print("Error: Unable to create the graph from the image.")
            return
        # Create a Graph object based on the adjacency matrix (if necessary)
        # graph = create_graph_from_adjacency_matrix(adjacency_matrix)
    else:
        print("Invalid choice. Please enter 'file' or 'image'.")
        return

    node_num = len(graph.nodes)
    seed_nodes = []  # Add code to input seed nodes if needed
    infection_prob = float(input("Enter the infection probability (0 to 1):"))
    if not 0 <= infection_prob <= 1:
        print("Invalid infection probability. Please enter a valid probability between 0 and 1.")
        return
    
    # Run independent cascade simulation
    independent_cascade(graph, seed_nodes, infection_prob, node_num)

    # Calculate data loss percentage and get data loss nodes
    total_nodes = node_num
    infected_nodes = sum(node.infected for node in graph.nodes)
    data_loss_nodes = [i for i, node in enumerate(graph.nodes) if not node.infected]
    data_loss_percentage = ((total_nodes - infected_nodes) / total_nodes) * 100.0

    print(f"Total Nodes: {total_nodes}")
    print(f"Infected Nodes: {infected_nodes}")
    print(f"Data Loss Percentage: {data_loss_percentage:.2f}%")
    print("Edges:")
    for i in range(node_num):
        for j in range(node_num):
            if graph.edges[i][j]:
                print(f"Edge: {i} -> {j}")


    # Visualize the graph with data loss nodes
    visualize_graph(graph, data_loss_nodes)

if __name__ == "_main_":
    main()

In [1]:
import cv2
import networkx as nx
import numpy as np
import random
import matplotlib.pyplot as plt

class Node:
    def _init_(self):
        self.infected = False

class Graph:
    def _init_(self, node_num):
        self.nodes = [Node() for _ in range(node_num)]
        self.edges = [[0 for _ in range(node_num)] for _ in range(node_num)]

def read_graph_from_file(file_name):
    with open(file_name, 'r') as file:
        lines = file.readlines()

    node_num = int(lines[0].strip())
    graph = Graph(node_num)

    for i in range(1, len(lines)):
        values = list(map(int, lines[i].strip().split()))
        for j in range(len(values)):
            graph.edges[i - 1][j] = values[j]

    return graph

def create_adjacency_matrix_from_image(image_path):
    try:
        print("Attempting to load image...")
        image = cv2.imread(image_path, 0)
        if image is None:
            print(f"Error: Unable to load image at {image_path}")
            return None, None

        cv2.imshow('Image', image)
        cv2.waitKey(0)
        _, thresholded = cv2.threshold(image, 200, 255, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        G = nx.Graph()

        for contour in contours:
            for i in range(len(contour)):
                node1 = tuple(contour[i][0])
                node2 = tuple(contour[(i + 1) % len(contour)][0])
                G.add_edge(node1, node2)

        adjacency_matrix = nx.adjacency_matrix(G).todense()
        return G, adjacency_matrix
    except Exception as e:
        print(f"Error: {e}")
        return None, None

def independent_cascade(graph, seed_nodes, infection_prob, node_num):
    new_infect = [0] * node_num

    for node in seed_nodes:
        graph.nodes[node].infected = True

    while True:
        any_new_infections = False

        for i in range(node_num):
            if graph.nodes[i].infected:
                for j in range(node_num):
                    if not graph.nodes[j].infected and graph.edges[i][j]:
                        random_value = random.random()
                        if random_value < infection_prob:
                            new_infect[j] = 1
                            any_new_infections = True

        if not any_new_infections:
            break

        for i in range(node_num):
            if new_infect[i]:
                graph.nodes[i].infected = True
                new_infect[i] = 0

def visualize_graph(graph, data_loss_nodes):
    G = nx.Graph(graph.edges)
    pos = nx.spring_layout(G)

    nx.draw(G, pos, with_labels=True, node_color='skyblue', node_size=1000, font_size=10, font_color='black')
    nx.draw_networkx_nodes(G, pos, nodelist=data_loss_nodes, node_color='red', node_size=1000)

    plt.title("Graph Visualization with Data Loss Nodes")
    plt.show()

def main():
    print("Starting main function...")
    file_or_image = input("Enter 'file' to read graph from a file or 'image' to read graph from an image: ")

    if file_or_image.lower() == 'file':
        file_name = input("Enter the name of the file containing the graph: ")
        graph = read_graph_from_file(file_name)
        if not graph:
            print("Error: Unable to create the graph. Please check the input file.")
            return
        
    elif file_or_image.lower() == 'image':
        image_path = input("Enter the path to the image containing the graph: ")
        graph, adjacency_matrix = create_adjacency_matrix_from_image(image_path)
        if graph is None or adjacency_matrix is None:
            print("Error: Unable to create the graph from the image.")
            return
        # Create a Graph object based on the adjacency matrix (if necessary)
        # graph = create_graph_from_adjacency_matrix(adjacency_matrix)
    else:
        print("Invalid choice. Please enter 'file' or 'image'.")
        return

    node_num = len(graph.nodes)
    seed_nodes = []  # Add code to input seed nodes if needed
    infection_prob = float(input("Enter the infection probability (0 to 1):"))
    if not 0 <= infection_prob <= 1:
        print("Invalid infection probability. Please enter a valid probability between 0 and 1.")
        return
    
    # Run independent cascade simulation
    independent_cascade(graph, seed_nodes, infection_prob, node_num)

    # Calculate data loss percentage and get data loss nodes
    total_nodes = node_num
    infected_nodes = sum(node.infected for node in graph.nodes)
    data_loss_nodes = [i for i, node in enumerate(graph.nodes) if not node.infected]
    data_loss_percentage = ((total_nodes - infected_nodes) / total_nodes) * 100.0

    print(f"Total Nodes: {total_nodes}")
    print(f"Infected Nodes: {infected_nodes}")
    print(f"Data Loss Percentage: {data_loss_percentage:.2f}%")
    print("Edges:")
    for i in range(node_num):
        for j in range(node_num):
            if graph.edges[i][j]:
                print(f"Edge: {i} -> {j}")


    # Visualize the graph with data loss nodes
    visualize_graph(graph, data_loss_nodes)

if __name__ == "_main_":
    main(

SyntaxError: incomplete input (1123908427.py, line 146)

In [3]:
import cv2
import networkx as nx
import numpy as np
import random
import matplotlib.pyplot as plt

class Node:
    def _init_(self):
        self.infected = False

class Graph:
    def _init_(self, node_num):
        self.nodes = [Node() for _ in range(node_num)]
        self.edges = [[0 for _ in range(node_num)] for _ in range(node_num)]

def read_graph_from_file(file_name):
    with open(file_name, 'r') as file:
        lines = file.readlines()

    node_num = int(lines[0].strip())
    graph = Graph(node_num)

    for i in range(1, len(lines)):
        values = list(map(int, lines[i].strip().split()))
        for j in range(len(values)):
            graph.edges[i - 1][j] = values[j]

    return graph

def create_adjacency_matrix_from_image(image_path):
    try:
        print("Attempting to load image...")
        image = cv2.imread(image_path, 0)
        if image is None:
            print(f"Error: Unable to load image at {image_path}")
            return None, None

        cv2.imshow('Image', image)
        cv2.waitKey(0)
        _, thresholded = cv2.threshold(image, 200, 255, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        G = nx.Graph()

        for contour in contours:
            for i in range(len(contour)):
                node1 = tuple(contour[i][0])
                node2 = tuple(contour[(i + 1) % len(contour)][0])
                G.add_edge(node1, node2)

        adjacency_matrix = nx.adjacency_matrix(G).todense()
        return G, adjacency_matrix
    except Exception as e:
        print(f"Error: {e}")
        return None, None

def independent_cascade(graph, seed_nodes, infection_prob, node_num):
    new_infect = [0] * node_num

    for node in seed_nodes:
        graph.nodes[node].infected = True

    while True:
        any_new_infections = False

        for i in range(node_num):
            if graph.nodes[i].infected:
                for j in range(node_num):
                    if not graph.nodes[j].infected and graph.edges[i][j]:
                        random_value = random.random()
                        if random_value < infection_prob:
                            new_infect[j] = 1
                            any_new_infections = True

        if not any_new_infections:
            break

        for i in range(node_num):
            if new_infect[i]:
                graph.nodes[i].infected = True
                new_infect[i] = 0

def visualize_graph(graph, data_loss_nodes):
    G = nx.Graph(graph.edges)
    pos = nx.spring_layout(G)

    nx.draw(G, pos, with_labels=True, node_color='skyblue', node_size=1000, font_size=10, font_color='black')
    nx.draw_networkx_nodes(G, pos, nodelist=data_loss_nodes, node_color='red', node_size=1000)

    plt.title("Graph Visualization with Data Loss Nodes")
    plt.show()

def main():
    print("Starting main function...")
    file_or_image = input("Enter 'file' to read graph from a file or 'image' to read graph from an image: ")

    if file_or_image.lower() == 'file':
        file_name = input("Enter the name of the file containing the graph: ")
        graph = read_graph_from_file(file_name)
        if not graph:
            print("Error: Unable to create the graph. Please check the input file.")
            return
        
    elif file_or_image.lower() == 'image':
        image_path = input("Enter the path to the image containing the graph: ")
        graph, adjacency_matrix = create_adjacency_matrix_from_image(image_path)
        if graph is None or adjacency_matrix is None:
            print("Error: Unable to create the graph from the image.")
            return
        # Create a Graph object based on the adjacency matrix (if necessary)
        # graph = create_graph_from_adjacency_matrix(adjacency_matrix)
    else:
        print("Invalid choice. Please enter 'file' or 'image'.")
        return

    node_num = len(graph.nodes)
    seed_nodes = []  # Add code to input seed nodes if needed
    infection_prob = float(input("Enter the infection probability (0 to 1):"))
    if not 0 <= infection_prob <= 1:
        print("Invalid infection probability. Please enter a valid probability between 0 and 1.")
        return
    
    # Run independent cascade simulation
    independent_cascade(graph, seed_nodes, infection_prob, node_num)

    # Calculate data loss percentage and get data loss nodes
    total_nodes = node_num
    infected_nodes = sum(node.infected for node in graph.nodes)
    data_loss_nodes = [i for i, node in enumerate(graph.nodes) if not node.infected]
    data_loss_percentage = ((total_nodes - infected_nodes) / total_nodes) * 100.0

    print(f"Total Nodes: {total_nodes}")
    print(f"Infected Nodes: {infected_nodes}")
    print(f"Data Loss Percentage: {data_loss_percentage:.2f}%")
    print("Edges:")
    for i in range(node_num):
        for j in range(node_num):
            if graph.edges[i][j]:
                print(f"Edge: {i} -> {j}")


    # Visualize the graph with data loss nodes
    visualize_graph(graph, data_loss_nodes)

if __name__ == "__main__":
    main()

Starting main function...
Enter 'file' to read graph from a file or 'image' to read graph from an image: image
Enter the path to the image containing the graph: "C:\Abishek\ex.jpg"
Attempting to load image...
Error: Unable to load image at "C:\Abishek\ex.jpg"
Error: Unable to create the graph from the image.
